In [1]:
import numpy as np
from keras.datasets import fashion_mnist as fmn
from keras.layers import Dense, Conv2D, Activation
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam, Nadam
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
import tensorflow as tf
from time import time

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Problem
Build a fully connected neural network in Keras to classify the images in Fashion-MNIST using one hidden layer of 5 neurons and a softmax output layer.  So the structure is 

    x_1             a_1
    x_2             a_2
    x_3             a_3
    .               a_4                softmax
    .               a_5
    .
    .
    x_784
    
where every $x_i$ is connected to every $a_i$, and every $a_i$ is connected to the softmax. Each $x_i$ corresponds to one pixel in an input image and each $a_i$ is a "hidden" neuron.  (note that the fashion-MNIST dataset is available in Keras: 

In [2]:
(x_train, y_train), (x_test, y_test) = fmn.load_data()

In [3]:
def build_keras_model(optimizer,activ) :
    output_dim = 10
    model = Sequential()
    model.add(Dense(5, input_dim=input_dim, activation=activ))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    return model

# Generate optimizers for each run
def generate_optimizers(learning_rate=.01):
    """Generate a dictionary of optimizers for keras to use in gridsearch"""
    sgd = SGD()
    momentum = SGD(lr=learning_rate, momentum=.9)
    nesterov = SGD(lr=learning_rate, momentum=.9, nesterov=True)
    rmsprop = RMSprop(lr=learning_rate)
    adagrad = Adagrad(lr=learning_rate)
    adadelta = Adadelta(lr=learning_rate)
    adam = Adam(lr=learning_rate)
    nadam = Nadam(lr=learning_rate)
    optimizer_dict = {'SGD':sgd,
                      'Momentum':momentum,
                      'Nesterov':nesterov, 
                      'RMSprop':rmsprop, 
                      'AdaGrad':adagrad, 
                      'AdaDelta':adadelta, 
                      'Adam':adam,
                      'Nadam':nadam}
    return optimizer_dict

1. Build and train the network using ReLU as the activation function for the hidden layer.
2. Build and train the network again using sigmoid as the activation function for the hidden layer. 

In [4]:
rates = [0.0001,0.001,.01,.1,1]
meth_list = ['SGD','Momentum','Nesterov','RMSprop','AdaGrad','AdaDelta','Adam','Nadam']

# Reshape y as one-hot
y_train = np_utils.to_categorical(y_train, 10).astype(float)
y_test = np_utils.to_categorical(y_test, 10).astype(float)

# Flatten images
input_dim = x_train.shape[1]*x_train.shape[2]
x_train = x_train.reshape((x_train.shape[0], input_dim))
x_test = x_test.reshape((x_test.shape[0], input_dim))
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

# Permute the training data to be in random order
perm = np.random.permutation(x_train.shape[0])
x_train = x_train[perm]
y_train = y_train[perm]

/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
relu_times = []
sigm_times = []

relu_acc = []
sigm_acc = []

for rate in rates :
    methods = generate_optimizers(rate)
    for meth in methods.keys() :
        method = methods[meth]
        model1 = build_keras_model(method,'relu')
        model2 = build_keras_model(method,'sigmoid')
        start = time()
        hist1 = model1.fit(x_train,
                y_train,
                batch_size=128,
                epochs=20,
                verbose=2,
                shuffle=False,
                validation_data=(x_test, y_test))
        end = time()
        relu_times.append(end-start)
        start = time()
        hist2 = model2.fit(x_train,
                y_train,
                batch_size=128,
                epochs=20,
                verbose=2,
                shuffle=False,
                validation_data=(x_test, y_test))
        end = time()
        sigm_times.append(end-start)
        relu_acc.append(max(hist1.history['acc']))
        sigm_acc.append(max(hist2.history['acc']))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 - 2s - loss: 1.2683 - acc: 0.5673 - val_loss: 0.9412 - val_acc: 0.7087
Epoch 2/20
 - 1s - loss: 0.8151 - acc: 0.7344 - val_loss: 0.7330 - val_acc: 0.7545
Epoch 3/20
 - 1s - loss: 0.6638 - acc: 0.7662 - val_loss: 0.6571 - val_acc: 0.7677
Epoch 4/20
 - 1s - loss: 0.6149 - acc: 0.7812 - val_loss: 0.6278 - val_acc: 0.7792
Epoch 5/20
 - 1s - loss: 0.5902 - acc: 0.7907 - val_loss: 0.6090 - val_acc: 0.7848
Epoch 6/20
 - 1s - loss: 0.5730 - acc: 0.7978 - val_loss: 0.5952 - val_acc: 0.7889
Epoch 7/20
 - 1s - loss: 0.5598 - acc: 0.8034 - val_loss: 0.5843 - val_acc: 0.7936
Epoch 8/20
 - 1s - loss: 0.5490 - acc: 0.8078 - val_loss: 0.5752 - val_acc: 0.7969
Epoch 9/20
 - 1s - loss: 0.5399 - acc: 0.8115 - val_loss: 0.5677 - val_acc: 0.8002
Epoch 10/20
 - 1s - loss: 0.5321 - acc: 0.8143 - val_loss: 0.5611 - val_acc: 0.8024
Epoch 11/20
 - 1s - loss: 0.5252 - acc: 0.8165 - val_loss: 0.5553 - val_acc: 0.8036
Epoch 12/20
 - 1s - loss: 0.5191 - 

 - 1s - loss: 0.8579 - acc: 0.6817 - val_loss: 0.8662 - val_acc: 0.6812
Epoch 17/20
 - 1s - loss: 0.8420 - acc: 0.6956 - val_loss: 0.8510 - val_acc: 0.6966
Epoch 18/20
 - 1s - loss: 0.8259 - acc: 0.7132 - val_loss: 0.8354 - val_acc: 0.7190
Epoch 19/20
 - 1s - loss: 0.8091 - acc: 0.7297 - val_loss: 0.8190 - val_acc: 0.7299
Epoch 20/20
 - 1s - loss: 0.7912 - acc: 0.7408 - val_loss: 0.8017 - val_acc: 0.7391
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 - 2s - loss: 2.3071 - acc: 0.1517 - val_loss: 2.2197 - val_acc: 0.1782
Epoch 2/20
 - 1s - loss: 2.1694 - acc: 0.1878 - val_loss: 2.1313 - val_acc: 0.1939
Epoch 3/20
 - 1s - loss: 2.1016 - acc: 0.2044 - val_loss: 2.0780 - val_acc: 0.2177
Epoch 4/20
 - 1s - loss: 2.0552 - acc: 0.2339 - val_loss: 2.0374 - val_acc: 0.2464
Epoch 5/20
 - 1s - loss: 2.0181 - acc: 0.2714 - val_loss: 2.0034 - val_acc: 0.3031
Epoch 6/20
 - 1s - loss: 1.9865 - acc: 0.3373 - val_loss: 1.9739 - val_acc: 0.3661
Epoch 7/20
 - 1s - loss: 1.9585 - acc: 0.388

Epoch 12/20
 - 1s - loss: 2.0926 - acc: 0.3314 - val_loss: 2.0892 - val_acc: 0.3351
Epoch 13/20
 - 1s - loss: 2.0876 - acc: 0.3381 - val_loss: 2.0845 - val_acc: 0.3404
Epoch 14/20
 - 1s - loss: 2.0831 - acc: 0.3442 - val_loss: 2.0803 - val_acc: 0.3474
Epoch 15/20
 - 1s - loss: 2.0790 - acc: 0.3501 - val_loss: 2.0763 - val_acc: 0.3537
Epoch 16/20
 - 1s - loss: 2.0751 - acc: 0.3556 - val_loss: 2.0726 - val_acc: 0.3591
Epoch 17/20
 - 1s - loss: 2.0715 - acc: 0.3605 - val_loss: 2.0692 - val_acc: 0.3630
Epoch 18/20
 - 1s - loss: 2.0681 - acc: 0.3653 - val_loss: 2.0660 - val_acc: 0.3680
Epoch 19/20
 - 1s - loss: 2.0649 - acc: 0.3694 - val_loss: 2.0629 - val_acc: 0.3711
Epoch 20/20
 - 1s - loss: 2.0618 - acc: 0.3730 - val_loss: 2.0600 - val_acc: 0.3748
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 - 2s - loss: 2.7373 - acc: 0.1492 - val_loss: 2.7373 - val_acc: 0.1476
Epoch 2/20
 - 1s - loss: 2.7260 - acc: 0.1497 - val_loss: 2.7255 - val_acc: 0.1476
Epoch 3/20
 - 1s - loss: 2.7

 - 1s - loss: 0.8373 - acc: 0.7079 - val_loss: 0.8349 - val_acc: 0.7066
Epoch 8/20
 - 1s - loss: 0.7820 - acc: 0.7243 - val_loss: 0.7850 - val_acc: 0.7417
Epoch 9/20
 - 1s - loss: 0.7348 - acc: 0.7676 - val_loss: 0.7434 - val_acc: 0.7618
Epoch 10/20
 - 1s - loss: 0.6938 - acc: 0.7742 - val_loss: 0.7062 - val_acc: 0.7673
Epoch 11/20
 - 1s - loss: 0.6584 - acc: 0.7821 - val_loss: 0.6757 - val_acc: 0.7739
Epoch 12/20
 - 1s - loss: 0.6321 - acc: 0.7866 - val_loss: 0.6530 - val_acc: 0.7792
Epoch 13/20
 - 1s - loss: 0.6126 - acc: 0.7904 - val_loss: 0.6361 - val_acc: 0.7824
Epoch 14/20
 - 1s - loss: 0.5972 - acc: 0.7930 - val_loss: 0.6226 - val_acc: 0.7841
Epoch 15/20
 - 1s - loss: 0.5845 - acc: 0.7951 - val_loss: 0.6114 - val_acc: 0.7881
Epoch 16/20
 - 1s - loss: 0.5737 - acc: 0.7973 - val_loss: 0.6018 - val_acc: 0.7911
Epoch 17/20
 - 1s - loss: 0.5644 - acc: 0.7995 - val_loss: 0.5933 - val_acc: 0.7932
Epoch 18/20
 - 1s - loss: 0.5562 - acc: 0.8021 - val_loss: 0.5860 - val_acc: 0.7961
Epoch 

Epoch 3/20
 - 1s - loss: 1.6689 - acc: 0.5815 - val_loss: 1.6022 - val_acc: 0.6117
Epoch 4/20
 - 1s - loss: 1.5421 - acc: 0.6304 - val_loss: 1.4872 - val_acc: 0.6414
Epoch 5/20
 - 1s - loss: 1.4357 - acc: 0.6561 - val_loss: 1.3908 - val_acc: 0.6626
Epoch 6/20
 - 1s - loss: 1.3460 - acc: 0.6723 - val_loss: 1.3096 - val_acc: 0.6757
Epoch 7/20
 - 1s - loss: 1.2701 - acc: 0.6862 - val_loss: 1.2408 - val_acc: 0.6869
Epoch 8/20
 - 1s - loss: 1.2053 - acc: 0.6989 - val_loss: 1.1821 - val_acc: 0.6975
Epoch 9/20
 - 1s - loss: 1.1498 - acc: 0.7105 - val_loss: 1.1319 - val_acc: 0.7092
Epoch 10/20
 - 1s - loss: 1.1020 - acc: 0.7197 - val_loss: 1.0885 - val_acc: 0.7164
Epoch 11/20
 - 1s - loss: 1.0605 - acc: 0.7279 - val_loss: 1.0508 - val_acc: 0.7241
Epoch 12/20
 - 1s - loss: 1.0241 - acc: 0.7352 - val_loss: 1.0177 - val_acc: 0.7329
Epoch 13/20
 - 1s - loss: 0.9918 - acc: 0.7432 - val_loss: 0.9880 - val_acc: 0.7398
Epoch 14/20
 - 1s - loss: 0.9626 - acc: 0.7501 - val_loss: 0.9609 - val_acc: 0.7455

...

 - 1s - loss: 0.5078 - acc: 0.8378 - val_loss: 0.5841 - val_acc: 0.8119
Epoch 8/20
 - 1s - loss: 0.5019 - acc: 0.8394 - val_loss: 0.5805 - val_acc: 0.8129
Epoch 9/20
 - 1s - loss: 0.4967 - acc: 0.8409 - val_loss: 0.5777 - val_acc: 0.8137
Epoch 10/20
 - 1s - loss: 0.4919 - acc: 0.8426 - val_loss: 0.5750 - val_acc: 0.8142
Epoch 11/20
 - 1s - loss: 0.4883 - acc: 0.8437 - val_loss: 0.5735 - val_acc: 0.8147
Epoch 12/20
 - 1s - loss: 0.4843 - acc: 0.8450 - val_loss: 0.5717 - val_acc: 0.8151
Epoch 13/20
 - 1s - loss: 0.4812 - acc: 0.8454 - val_loss: 0.5713 - val_acc: 0.8152
Epoch 14/20
 - 1s - loss: 0.4780 - acc: 0.8461 - val_loss: 0.5708 - val_acc: 0.8151
Epoch 15/20
 - 1s - loss: 0.4749 - acc: 0.8473 - val_loss: 0.5699 - val_acc: 0.8153
Epoch 16/20
 - 1s - loss: 0.4723 - acc: 0.8482 - val_loss: 0.5687 - val_acc: 0.8155
Epoch 17/20
 - 1s - loss: 0.4704 - acc: 0.8488 - val_loss: 0.5686 - val_acc: 0.8159
Epoch 18/20
 - 1s - loss: 0.4682 - acc: 0.8496 - val_loss: 0.5680 - val_acc: 0.8154
Epoch 

Epoch 3/20
 - 1s - loss: 2.1145 - acc: 0.4732 - val_loss: 2.3457 - val_acc: 0.4828
Epoch 4/20
 - 1s - loss: 2.1307 - acc: 0.4682 - val_loss: 2.4811 - val_acc: 0.4691
Epoch 5/20
 - 1s - loss: 2.1728 - acc: 0.4642 - val_loss: 2.1365 - val_acc: 0.4595
Epoch 6/20
 - 1s - loss: 2.0378 - acc: 0.4705 - val_loss: 2.2579 - val_acc: 0.4440
Epoch 7/20
 - 1s - loss: 2.2386 - acc: 0.4718 - val_loss: 2.2739 - val_acc: 0.4748
Epoch 8/20
 - 1s - loss: 2.3690 - acc: 0.4639 - val_loss: 2.3630 - val_acc: 0.4918
Epoch 9/20
 - 1s - loss: 2.2439 - acc: 0.4956 - val_loss: 2.1810 - val_acc: 0.4880
Epoch 10/20
 - 1s - loss: 2.3108 - acc: 0.4680 - val_loss: 2.1845 - val_acc: 0.4594
Epoch 11/20
 - 1s - loss: 2.3357 - acc: 0.4565 - val_loss: 2.7593 - val_acc: 0.4278
Epoch 12/20
 - 1s - loss: 2.5821 - acc: 0.4760 - val_loss: 2.1284 - val_acc: 0.4690
Epoch 13/20
 - 1s - loss: 2.2634 - acc: 0.4674 - val_loss: 2.2737 - val_acc: 0.4678
Epoch 14/20
 - 1s - loss: 2.3021 - acc: 0.4384 - val_loss: 2.3167 - val_acc: 0.4541

Epoch 19/20
 - 1s - loss: 11.1748 - acc: 0.2615 - val_loss: 11.2157 - val_acc: 0.2570
Epoch 20/20
 - 1s - loss: 11.1748 - acc: 0.2615 - val_loss: 11.2157 - val_acc: 0.2570
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 - 3s - loss: 0.8565 - acc: 0.6990 - val_loss: 0.7286 - val_acc: 0.7593
Epoch 2/20
 - 1s - loss: 0.7113 - acc: 0.7632 - val_loss: 0.7495 - val_acc: 0.7540
Epoch 3/20
 - 1s - loss: 0.6623 - acc: 0.7820 - val_loss: 0.7308 - val_acc: 0.7626
Epoch 4/20
 - 1s - loss: 0.6469 - acc: 0.7942 - val_loss: 0.6767 - val_acc: 0.7886
Epoch 5/20
 - 1s - loss: 0.6400 - acc: 0.7951 - val_loss: 0.6808 - val_acc: 0.7874
Epoch 6/20
 - 1s - loss: 0.6349 - acc: 0.7924 - val_loss: 0.6449 - val_acc: 0.7924
Epoch 7/20
 - 1s - loss: 0.6229 - acc: 0.8009 - val_loss: 0.6793 - val_acc: 0.7832
Epoch 8/20
 - 1s - loss: 0.6269 - acc: 0.7998 - val_loss: 0.6528 - val_acc: 0.8019
Epoch 9/20
 - 1s - loss: 0.6156 - acc: 0.8097 - val_loss: 0.6364 - val_acc: 0.8065
Epoch 10/20
 - 1s - loss: 0.610

 - 1s - loss: 1.4523 - acc: 0.6049 - val_loss: 1.3265 - val_acc: 0.6347
Epoch 15/20
 - 1s - loss: 1.4396 - acc: 0.6108 - val_loss: 1.3865 - val_acc: 0.5401
Epoch 16/20
 - 1s - loss: 1.4596 - acc: 0.6095 - val_loss: 1.2104 - val_acc: 0.6697
Epoch 17/20
 - 1s - loss: 1.4600 - acc: 0.6027 - val_loss: 1.5498 - val_acc: 0.5562
Epoch 18/20
 - 1s - loss: 1.4565 - acc: 0.6021 - val_loss: 1.2488 - val_acc: 0.6575
Epoch 19/20
 - 1s - loss: 1.5719 - acc: 0.5928 - val_loss: 1.3138 - val_acc: 0.5180
Epoch 20/20
 - 1s - loss: 1.4155 - acc: 0.6180 - val_loss: 1.1401 - val_acc: 0.6554
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 - 3s - loss: 8.6073 - acc: 0.2367 - val_loss: 6.3347 - val_acc: 0.2905
Epoch 2/20
 - 1s - loss: 6.0844 - acc: 0.2834 - val_loss: 6.0241 - val_acc: 0.2827
Epoch 3/20
 - 1s - loss: 5.9370 - acc: 0.2822 - val_loss: 5.9341 - val_acc: 0.2838
Epoch 4/20
 - 1s - loss: 5.9079 - acc: 0.2812 - val_loss: 5.9079 - val_acc: 0.2850
Epoch 5/20
 - 1s - loss: 5.8764 - acc: 0.2

 - 1s - loss: 11.1677 - acc: 0.1633 - val_loss: 11.1625 - val_acc: 0.1697
Epoch 10/20
 - 1s - loss: 11.1678 - acc: 0.1633 - val_loss: 11.1626 - val_acc: 0.1697
Epoch 11/20
 - 1s - loss: 11.1678 - acc: 0.1633 - val_loss: 11.1627 - val_acc: 0.1697
Epoch 12/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.1628 - val_acc: 0.1697
Epoch 13/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.1628 - val_acc: 0.1697
Epoch 14/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.1628 - val_acc: 0.1697
Epoch 15/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.1628 - val_acc: 0.1697
Epoch 16/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.1628 - val_acc: 0.1697
Epoch 17/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.1628 - val_acc: 0.1697
Epoch 18/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.1628 - val_acc: 0.1697
Epoch 19/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.1628 - val_acc: 0.1697
Epoch 20/20
 - 1s - loss: 11.1679 - acc: 0.1633 - val_loss: 11.162

In [16]:
print('\tRate\tMethod\tTime\t\t\tAccuracy')
n = len(relu_acc)
for i in range(n) :
    if i == 16 :
        print('relu:\t'+str(rates[i//(n//5)])+'\t'+str(meth_list[i%8][:5])+'\t'+str(relu_times[i])+'\t\t'+str(relu_acc[i]))
    else :
        print('relu:\t'+str(rates[i//(n//5)])+'\t'+str(meth_list[i%8][:5])+'\t'+str(relu_times[i])+'\t'+str(relu_acc[i]))
    print('sigm:\t'+str(rates[i//(n//5)])+'\t'+str(meth_list[i%8][:5])+'\t'+str(sigm_times[i])+'\t'+str(sigm_acc[i]))

	Rate	Method	Time			Accuracy
relu:	0.0001	SGD	17.17855930328369	0.8291833333651225
sigm:	0.0001	SGD	16.875957250595093	0.7923166666984558
relu:	0.0001	Momen	16.976959466934204	0.7309999999682109
sigm:	0.0001	Momen	16.81363606452942	0.5621666666348776
relu:	0.0001	Neste	16.97317910194397	0.7407833333333333
sigm:	0.0001	Neste	16.935710430145264	0.5805166666984558
relu:	0.0001	RMSpr	17.089823246002197	0.8102333333651225
sigm:	0.0001	RMSpr	17.222754955291748	0.7693666666666666
relu:	0.0001	AdaGr	17.120349884033203	0.4037833333492279
sigm:	0.0001	AdaGr	17.26282787322998	0.37298333331743877
relu:	0.0001	AdaDe	17.387486696243286	0.16078333332935968
sigm:	0.0001	AdaDe	17.95742392539978	0.1024
relu:	0.0001	Adam	19.944482564926147	0.8162000000317892
sigm:	0.0001	Adam	20.799670219421387	0.7271499999682108
relu:	0.0001	Nadam	21.16041135787964	0.8079333333015442
sigm:	0.0001	Nadam	18.565675020217896	0.76345
relu:	0.001	SGD	17.564343214035034	0.8311500000317892
sigm:	0.001	SGD	17.56078600883484	0.77

Compare the training time and the accuracy of your two neural network to each other and to the training time and accuracy of the XGBoost classifier you built previously. 

In [19]:
import xgboost as xgb
(x_train, y_train), (x_test, y_test) = fmn.load_data()
input_dim = x_train.shape[1]*x_train.shape[2]
x_train = x_train.reshape((x_train.shape[0], input_dim))
x_test = x_test.reshape((x_test.shape[0], input_dim))
gamma = 0.1
learning_rate = 0.9
clf = xgb.XGBClassifier(gamma=gamma,learning_rate=learning_rate)
start = time()
clf.fit(x_train,y_train)
end = time()
T = end - start

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f36b878ada0>>
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 368, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


TypeError: score() missing 1 required positional argument: 'y'

In [22]:
acc = clf.score(x_test,y_test)
print('\tTime\t\t\tAccuracy')
print('XGBoost:'+str(T)+'\t'+str(acc))

	Time			Accuracy
XGBoost:1118.374451637268	0.8869


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
